In [39]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import string

# Read recipe inputs
ACCOUNTS_WITH_BUNDLER_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_BUNDLER_AND_DUNS")
ACCOUNTS_WITH_BUNDLER_AND_DUNS_df = ACCOUNTS_WITH_BUNDLER_AND_DUNS.get_dataframe()
df = ACCOUNTS_WITH_BUNDLER_AND_DUNS_df

/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:413: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38488), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38490), raddr=('10.92.106.166', 10001)>
  if __name__ == '__main__':


In [40]:
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME
0,6121005966569,HEIDLER ROOFING SERVICES INC,HEIDLER ROOFING SERVICES INC,152141495.0,False,152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc."
1,9100001804959,HEIDLER ROOFING SERVICES INC,NaN,152141495.0,True,152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc.",152141495.0,"Heidler Roofing Services, Inc."
2,9100006532335,ANDERSON HOMES,NaN,608585766.0,True,608585766.0,Anderson Homes,608585766.0,Anderson Homes,608585766.0,Anderson Homes,608585766.0,Anderson Homes
3,9100008971857,ARARAT TRADING INC,NaN,255895117.0,True,NaN,NaN,NaN,NaN,NaN,NaN,255895117.0,Ararat Trading (2002) Corp.
4,9100005816790,ANGELS TAXI LLC,NaN,24832690.0,True,NaN,NaN,NaN,NaN,NaN,NaN,24832690.0,Angel's Taxi LLC


In [41]:
import warnings
warnings.filterwarnings(action='once')

df['DUNS'] = df['DUNS'].astype('Int64', errors='ignore')
df['DNB_DUNS_NUMBER'] = df['DNB_DUNS_NUMBER'].astype('Int64', errors='ignore')
df['DNB_BUSINESS_NAME'] = df['DNB_BUSINESS_NAME'].str.upper()
df["DNB_BUSINESS_NAME"] = df['DNB_BUSINESS_NAME'].str.replace('[^\w\s]','')

df['DNB_GLOBAL_ULT_NUMBER'] = df['DNB_GLOBAL_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_GLOBAL_ULT_NAME'] = df['DNB_GLOBAL_ULT_NAME'].str.upper()
df["DNB_GLOBAL_ULT_NAME"] = df['DNB_GLOBAL_ULT_NAME'].str.replace('[^\w\s]','')

df['DNB_DOMESTIC_ULT_NUMBER'] = df['DNB_DOMESTIC_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_DOMESTIC_ULTIMATE_NAME'] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.upper()
df["DNB_DOMESTIC_ULTIMATE_NAME"] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.replace('[^\w\s]','')

df['DNB_HQ_NUMBER'] = df['DNB_HQ_NUMBER'].astype('Int64', errors='ignore')
df['DNB_HQ_NAME'] = df['DNB_HQ_NAME'].str.upper()
df["DNB_HQ_NAME"] = df['DNB_HQ_NAME'].str.replace('[^\w\s]','')

df['DNB_LEVEL'] = 'None'
df['DNB_CUSTOMER_NAME'] = np.nan

df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_LEVEL'] = "DUNS Global"
df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_CUSTOMER_NAME'] = df.DNB_GLOBAL_ULT_NAME

df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_LEVEL'] = "DUNS Domestic"
df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_DOMESTIC_ULTIMATE_NAME

df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_LEVEL'] = "DUNS HQ"
df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_HQ_NAME

df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_LEVEL'] = "DUNS"
df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_BUSINESS_NAME

df['EDW_STATE'] = 'Unknown'
df.loc[df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "None"
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "Set"

df['CUSTOMER'] = np.nan
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["DNB_CUSTOMER_NAME"]
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["CUSTOMER_ACCOUNT_NAME"]

df.CUSTOMER.value_counts(dropna=False)

<>:7: DeprecationWarning: invalid escape sequence \w


OREILLY AUTOMOTIVE INC             6314
COMMONWEALTH OF PENNSYLVANIA       2426
RENTACENTER                        2173
STATE OF NEW YORK                  1838
FERGUSON ENTERPRISES               1776
                                   ... 
NUNEZ CONCRETE  MASONRY INC           1
EXELON BUSINESS SERV 5EF8             1
PUBLIC WATER SUPPLY DISTRICT 7        1
WASHINGTON MAINTENANCE SERVICES       1
RIVERLAND GARDEN APARTMENTS           1
Name: CUSTOMER, Length: 668537, dtype: int64

In [42]:
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = df

# Write recipe outputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.write_with_schema(ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df)

/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:437: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38540), raddr=('10.92.106.166', 10001)>
  return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:413: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38542), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)


844417 rows successfully written (gZ5yq3AH96)


/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:437: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38544), raddr=('10.92.106.166', 10001)>
  return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:425: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 38546), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_get_call(path, data, **kwargs), err_msg = err_msg)
